In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.decomposition import PCA
from scipy.stats import ttest_1samp, zscore, norm, chi2_contingency

df = pd.read_csv("/content/Student_Mental_Stress_and_Coping_Mechanisms.csv")

df = df.drop(columns=["Student ID"])

label_encoders = {}
for col in df.select_dtypes(include='object').columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le
df['Stress_Binary'] = (df['Mental Stress Level'] >= 6).astype(int)

X = df.drop(columns=["Mental Stress Level", "Stress_Binary"])
y_class = df["Stress_Binary"]
y_reg = df["Mental Stress Level"]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X_scaled, y_class, test_size=0.2, random_state=42)
X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(X_scaled, y_reg, test_size=0.2, random_state=42)

models = {
    "Logistic Regression": LogisticRegression(),
    "Random Forest": RandomForestClassifier(random_state=42),
    "SVM": SVC()
}

results = {}
for name, model in models.items():
    model.fit(X_train_c, y_train_c)
    y_pred = model.predict(X_test_c)
    results[name] = {
        "Accuracy": accuracy_score(y_test_c, y_pred),
        "Precision": precision_score(y_test_c, y_pred),
        "Recall": recall_score(y_test_c, y_pred),
        "F1-Score": f1_score(y_test_c, y_pred)
    }

print("\n=== Model Performance Comparison ===")
for name, metrics in results.items():
    print(f"\n{name}")
    for metric, value in metrics.items():
        print(f"{metric}: {value:.4f}")

rf = models["Random Forest"]
importances = rf.feature_importances_
features = X.columns
feature_importance = sorted(zip(features, importances), key=lambda x: x[1], reverse=True)
top_features = feature_importance[:3]

print("\n=== Top 3 Important Features from Random Forest ===")
for name, score in top_features:
    print(f"{name}: {score:.4f}")

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)
print("\nExplained Variance by PCA components:", pca.explained_variance_ratio_)

t_stat, p_val = ttest_1samp(df["Mental Stress Level"], popmean=5)
print("\n=== One-sample T-test ===")
print(f"T-statistic: {t_stat:.4f}, P-value: {p_val:.4f}")
if p_val < 0.05:
    print("Reject null hypothesis: Mean stress level significantly differs from 5.")
else:
    print("Fail to reject null hypothesis: No significant difference from mean = 5.")

group1 = df[df["Counseling Attendance"] == label_encoders["Counseling Attendance"].transform(["Yes"])[0]]["Study Hours Per Week"]
group2 = df[df["Counseling Attendance"] == label_encoders["Counseling Attendance"].transform(["No"])[0]]["Study Hours Per Week"]

mean_diff = group1.mean() - group2.mean()
std_error = np.sqrt(group1.var()/len(group1) + group2.var()/len(group2))
z_stat = mean_diff / std_error
p_z = 2 * (1 - norm.cdf(abs(z_stat)))

print("\n=== Z-test: Study Hours vs Counseling Attendance ===")
print(f"Z-statistic: {z_stat:.4f}, P-value: {p_z:.4f}")
if p_z < 0.05:
    print("Significant difference in study hours between counseling and non-counseling students.")
else:
    print("No significant difference in study hours.")

contingency = pd.crosstab(df["Gender"], df["Counseling Attendance"])
chi2_stat, chi2_p, _, _ = chi2_contingency(contingency)

print("\n=== Chi-square Test: Gender vs Counseling Attendance ===")
print(f"Chi2 Statistic: {chi2_stat:.4f}, P-value: {chi2_p:.4f}")
if chi2_p < 0.05:
    print("Significant relationship between gender and counseling attendance.")
else:
    print("No significant relationship between gender and counseling attendance.")



=== Model Performance Comparison ===

Logistic Regression
Accuracy: 0.4803
Precision: 0.4328
Recall: 0.4143
F1-Score: 0.4234

Random Forest
Accuracy: 0.5197
Precision: 0.4717
Recall: 0.3571
F1-Score: 0.4065

SVM
Accuracy: 0.5263
Precision: 0.4821
Recall: 0.3857
F1-Score: 0.4286

=== Top 3 Important Features from Random Forest ===
Study Hours Per Week: 0.1261
Physical Exercise (Hours per week): 0.0820
Age: 0.0771

Explained Variance by PCA components: [0.06839345 0.06683797]

=== One-sample T-test ===
T-statistic: 3.4489, P-value: 0.0006
Reject null hypothesis: Mean stress level significantly differs from 5.

=== Z-test: Study Hours vs Counseling Attendance ===
Z-statistic: 1.1756, P-value: 0.2397
No significant difference in study hours.

=== Chi-square Test: Gender vs Counseling Attendance ===
Chi2 Statistic: 8.9103, P-value: 0.2592
No significant relationship between gender and counseling attendance.
